## Timeseries Anomaly Detection with Moving Averages 

Megan O

Source: https://medium.com/@iliasfl/data-science-tricks-simple-anomaly-detection-for-metrics-with-a-weekly-pattern-2e236970d77

In [19]:
import math
import time
import random 

In [20]:

#exponential moving average
# where x = current real value measurement 
def ema(curVal, oldEma):
    w = 0.05 #w controls the influence of the new on the old 
    ema = w * oldEma + (1-w) * curVal 
    return ema


In [21]:
# exponential moving standard deviation
# EMS <- sqrt( w * EMS^2 + (1 - w) * (x - EMA)^2 )
def ems(curVal, ema, oldEms): 
    w = 0.05 
    temp = (w * oldEms*oldEms) + ((1-w) * (curVal - ema)*(curVal - ema))
    return math.sqrt(temp)


In [22]:
#initialize a fake initial timeseries, and the initial EMA and EMS 
ts = [91.4634, 91.4634, 91.4615, 91.1834, 91.8788, 91.1772, 89.7992, 90.3571, 92.0227, 91.9105, 91.141, 91.9571, 91.8744, 90.1645, 89.8047, 91.6464, 93.1426, 93.4519, 92.3365, 91.5574, 92.0669, 92.2745, 92.7285, 94.4017, 95.7123, 95.2184, 95.2685, 96.3945, 96.8731, 97.1613, 97.054, 97.7903, 98.6842, 98.6842, 98.8201, 99.1469, 99.3377, 99.3377, 99.3377, 99.1469, 98.9404, 99.0789, 98.9587, 99.3053, 99.7168, 99.4225, 99.7811, 101.133, 102.041, 101.372, 100.954, 101.278, 101.495, 101.839, 102.041, 102.041, 102.041, 102.041, 102.228, 102.578, 102.67, 102.32, 102.041, 102.041, 101.581, 101.059, 101.174, 101.166, 99.5098, 98.0392, 97.8279, 96.9769, 95.6878, 95.2898, 95.2657, 93.4809, 91.1667, 90.3614, 88.0565, 85.8787, 85.819, 84.4828, 83.5102, 83.5912, 83.184, 83.0755, 83.1042, 82.674, 82.6416, 82.6266, 82.1936, 82.2086, 81.7388, 80.637, 80.2139, 80.2139, 80.2415, 80.4571, 80.7847, 81.0026, 81.0811, 81.0811, 81.3063, 82.4324, 83.5102, 84.1379, 85.3978, 85.5231, 85.0443, 85.4237, 85.1719, 84.9153, 85.3747, 86.185, 86.7052, 86.8383, 87.0903, 87.2093, 87.2093, 87.3113, 87.5746, 87.9856, 88.2353, 88.8264, 90.721, 92.0245, 92.0245, 91.4768, 90.1074, 89.2857, 90.463, 91.9982, 92.3535, 93.6437, 93.229, 91.8115, 92.6347, 93.1677, 93.7255, 93.1451, 91.5578, 91.3269, 92.3565, 93.389, 93.9649, 94.7596, 95.6767, 96.7582, 97.6169, 97.9552, 98.1321, 98.4546, 98.6842, 98.6842, 98.6842, 98.6842, 98.7522, 99.0789, 99.6689, 100, 100, 100, 100.067, 100.403, 100.671, 100.949, 101.507, 101.757, 101.633, 101.823, 102.424, 103.068, 103.423, 103.233, 103.034, 103.151, 103.034, 103.134, 103.448, 103.448, 103.448, 103.618, 103.977, 104.167, 104.161, 103.802, 103.756, 103.862, 103.362, 103.034, 103.075, 102.565, 101.329, 100.96, 101.3, 100.711, 99.4639, 99.0789, 98.9927, 98.1459, 97.6987, 97.5524, 96.4537, 93.8084, 92.2608, 92.0937, 91.3595, 91.2461, 90.0043, 88.3902, 88.2833, 88.5444, 87.4915, 85.9699, 85.6912, 84.9689, 84.2697, 84.2697, 84.2697, 84.4506, 84.8552, 85.3966, 85.9468, 85.7898, 85.0731, 84.7458, 85.0944, 85.5787, 85.7143, 85.4903, 85.2468, 85.3837, 86.8877, 88.2353, 86.9463, 86.9463, 86.9463, 86.9463, 85.6331, 87.4806, 88.8293, 90.5285, 91.3636, 92.6799, 93.75, 94.2014, 95.1731, 95.9236, 96.464, 96.5385, 96.3871, 96.6228, 96.7842, 97.0984, 97.6572, 98.1613, 98.5475, 98.6378, 98.3153, 98.3359, 98.8885, 99.0789, 99.0331, 99.5263, 100, 99.1373, 99.2432, 100.944, 101.362, 99.8657, 98.6842, 98.9665, 98.4525, 97.4782, 97.6698, 98.0392, 97.1267, 97.3302, 98.892, 99.1404, 98.5695, 99.098, 100.295, 100.4, 100.105, 100, 99.9421, 99.2842, 98.9692, 99.2959, 99.0417, 98.3924, 98.0418, 98.3643, 98.1767, 97.8947, 98.4023, 98.6429, 98.3204, 98.6262, 99.5059, 99.4131, 98.7099, 98.8651, 99.3377, 99.058, 98.9404, 99.0789, 98.8698, 98.6842, 99.011, 99.0789, 98.9404, 99.0789, 98.9404, 99.0606, 99.0293, 98.7025, 98.5191, 98.1966, 98.0808, 98.73, 97.1986, 94.6995, 94.3396, 94.2072, 94.2072, 94.2072, 94.2072, 94.2072, 94.2072, 88.9888, 90.3422, 90.9091, 91.1685, 92.3431, 93.904, 94.7699, 95.2657, 96.5805, 98.0255, 98.9472, 100.067, 100.677, 101.361, 101.754, 101.633, 102.084, 102.555, 102.788, 103.142, 102.761, 101.782, 101.633, 101.757, 101.544, 101.376, 101.721, 101.732, 101.387, 101.351, 101.351, 101.297, 100.957, 101, 101.074, 100.35, 99.6098, 99.2802, 98.9535, 99.6379, 100.631, 100.478, 100.268, 100.524, 101.074, 101.351, 101.351, 101.569, 101.569, 101.569, 100.623, 99.6296, 99.6296, 104.167, 103.917, 103.75, 104.14, 104.671, 104.826, 104.102, 103.448, 103.448, 103.23, 102.876, 102.839, 103.151, 103.051, 103.077, 103.716]

emaInit = ts[0]
emsInit = 1 

for i in ts:
    emaInit = ema(i, emaInit)
    emsInit = ems(i, emaInit, emsInit)

In [23]:
print emaInit, emsInit #starting values 

103.683995567 0.0311961022164


In [27]:
# Alarm = abs(x - EMA) > n * EMS 
# generate x (curval) randomly inline 
def alarm(x, ema, ems): 
    n = 2 # number of standard deviations. the lower n is, the more false alarms we'll get. 
    if abs(x - ema) > n* ems:
        print "ALARM! x =" + str(x) 
    else:
        print "within normal, x =" + str(x)


In [25]:
def simulateAlarm(): 
    emaVal = emaInit 
    emsVal = emsInit
    numIter = 0 
    while True and numIter < 50:
        numIter += 1 
        x = random.uniform(50, 150)
        emaVal = ema(x, emaVal)
        emsVal = ems(x, emaVal, emsVal)
        alarm(x, emaVal, emsVal)
        time.sleep(1) 

In [28]:
simulateAlarm()

within normal, x =81.0749937832
within normal, x =81.7127078258
within normal, x =91.3314900188
within normal, x =85.8782431273
within normal, x =51.697287437
within normal, x =58.0758223417
within normal, x =104.354985321
within normal, x =59.1208888846
within normal, x =72.3762918253
within normal, x =66.8297897506
within normal, x =123.208449777
within normal, x =77.3499312824
within normal, x =142.019069294
within normal, x =112.867668686
within normal, x =114.173722198
within normal, x =120.341718057
within normal, x =133.582304631
within normal, x =112.889438333
within normal, x =134.537339573
within normal, x =65.2017594028
within normal, x =122.691833314
within normal, x =120.431775733
within normal, x =75.5761463011
within normal, x =68.2338867133
within normal, x =79.1444585145
within normal, x =51.1881783059
within normal, x =109.793177142
within normal, x =149.228253052
within normal, x =68.162241132
within normal, x =88.6291672003
within normal, x =118.594055995


KeyboardInterrupt: 